# Objective

As COVID-19 is wreaking havoc , every Data Scientist is trying to get some insights from related data. In this notebook I am also attempting to get answers for most important questions with the help of Data Visualization.

* **What we can learn from countries like China ,South Korea and Italy the most impacted countries**
* **What is the current state in countries where it started late , like India and US and what could be the possible future state in these  two countries**

The intent is to get basic insights without complex graphs and code , and to keep things simple and easy to understand I have used only Plotly **Scatter** and **Bar** graphs

Please leave comments as feedback and if your like the work please do upvote

# How China,Italy and South Korea is impacted

Let's start with importing required data set and verifying columns

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px
import plotly.graph_objs as go #for creating interactive graphs
from plotly.subplots import make_subplots
from datetime import datetime # for date time columns


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
#import main data set
df=pd.read_csv('../input/novel-corona-virus-2019-dataset/covid_19_data.csv')
print("Importing the data set and verifying the columns")

#Converting date column into correct format
df['ObservationDate']=pd.to_datetime(df['ObservationDate'])

#Geting latest timestamp
latest_timestamp=df.iloc[df.last_valid_index()]['ObservationDate']
#Geting latest date
latest_date=latest_timestamp.strftime("%Y-%m-%d")
print("Data available till {}".format(latest_date))

#Checking Columns in data set
df.head()

Let's add one more columns here , **Active Cases** (**Confirmed Cases-Recovered-Deaths**) 

In [ ]:
#Adding Active cases
df['ActiveCases'] = df['Confirmed'] - df['Deaths'] - df['Recovered']
print("Active Cases Column Added Successfully")
df.head()

To answer our first question let's start our analysis with 'China' and try to understand how virus spread across the period of time.

As data is present across provinces in China , we will grouped the data by Observation Date and add one more column 'Week' to see trend across the weeks

In [ ]:
#Taking out of data for China region
df_china=df[df['Country/Region'] == 'Mainland China']
print("Number of Records for China are {}".format(df_china.shape))
df_china.head()

In [ ]:
#Group by date
df_china=df_china.groupby(['ObservationDate']).sum().reset_index()
print("Grouped by Observation Date")
df_china['Week'] = df_china['ObservationDate'].dt.week
print("Week Column Added successfully")
df_china.head()

Let's visualize trends in China , first we will see individual trends

In [ ]:
#Visualizing Number of Confirmed cases , Deaths and Recovery in China
fig1=px.line(df_china,x='ObservationDate',y='Confirmed',title='Confirmed Cases in China')
fig2=px.line(df_china,x='ObservationDate',y='Deaths',title='Deaths in China')
fig3=px.line(df_china,x='ObservationDate',y='Recovered',title='Recovery in China')
fig4=px.line(df_china,x='ObservationDate',y='ActiveCases',title='Active Cases in China')

fig1.show()
fig2.show()
fig3.show()
fig4.show()

> **There is sudden spike in Deaths in the month of April , these numbers are the missing one which was not reported initially**

Let's visualize this in single graph for comparision

In [ ]:
#Visualizing Cumaltive trends
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_china['ObservationDate'], y=df_china['Confirmed'],
                    mode='lines',name=' Confirmed Cases'))
fig.add_trace(go.Scatter(x=df_china['ObservationDate'], y=df_china['Deaths'], 
                mode='lines',name='Deaths'))
fig.add_trace(go.Scatter(x=df_china['ObservationDate'], y=df_china['Recovered'], 
                mode='lines',name='Recovered Cases'))
fig.add_trace(go.Scatter(x=df_china['ObservationDate'], y=df_china['ActiveCases'], 
                mode='lines',name='Active Cases'))

        
    
fig.update_layout(title_text='Trend in China',plot_bgcolor='rgb(250, 242, 242)')

fig.show()

In [ ]:
#Getting the latest observation date from data set
df.iloc[df.last_valid_index()]['ObservationDate']
#Taking out record for latest records in seprate data frame
df_china_lately=df_china[df_china['ObservationDate'] == latest_date]
#Creating temporary data frame for visualizing purpose
df_china_status=pd.DataFrame(columns=['Numbers','Status'])
#Appending Records from latest data frame
df_china_status=df_china_status.append({'Numbers':df_china_lately.iloc[0]['Confirmed'],'Status':'Confirmed'},ignore_index=True)
df_china_status=df_china_status.append({'Numbers':df_china_lately.iloc[0]['ActiveCases'],'Status':'Active Cases'},ignore_index=True)
df_china_status=df_china_status.append({'Numbers':df_china_lately.iloc[0]['Recovered'],'Status':'Recovered'},ignore_index=True)
df_china_status=df_china_status.append({'Numbers':df_china_lately.iloc[0]['Deaths'],'Status':'Deaths'},ignore_index=True)
#Visualizing graphs
fig = px.bar(df_china_status, y='Numbers', x='Status',title='Latest Status in China' , color='Numbers')
fig.show()

**📌Take away points**

* Number of confirmed cases surged in January last week and mid of Feburary but from March first week there is not sudden rise in confirmed cases , it's kind of stable after that
* Number of Deaths are increasing through out in the month of Feb and March , it's keep on increasing when number of confirmed cases are not increasing much this may be becuase previously confirmed cases were not able to recovered
* Number of recovery is pretty good , from Feburary last week there is suddent rise in recovered cases and that's why active cases are very less . China did a pretty good job in fighting back

In [ ]:
#Calculating Death,Active Cases and Recovery percentage
death_percentage=round((df_china_lately.iloc[0]['Deaths']/df_china_lately.iloc[0]['Confirmed'])*100)
active_cases_percentage=round((df_china_lately.iloc[0]['ActiveCases']/df_china_lately.iloc[0]['Confirmed'])*100)
recovery_percentage=round((df_china_lately.iloc[0]['Recovered']/df_china_lately.iloc[0]['Confirmed'])*100)

print("In China Active Cases and Recovery percentage is {} ,{} respectively and Death rate is {} ".format(active_cases_percentage,recovery_percentage,death_percentage))

>Well , this is a tremendous comeback for China.After seeing rapid surge in confirmed cases Chinese goverment were able to control this panedemic in short period of time.They are following an ideal graph for Active and Recovered cases, any country who is fighting with COVID-19 should exhibit one of these characterstics

Let's start looking the numbers in Italy

In [ ]:
#Taking out of data for italy region
df_italy=df[df['Country/Region'] == 'Italy']
print("Number of Records for Italy are {}".format(df_italy.shape))
df_italy.head()

Let's visualize trends in Italy , first we will see individual trends

In [ ]:
#Group by date
df_italy=df_italy.groupby(['ObservationDate']).sum().reset_index()
df_italy['Week'] = df_italy['ObservationDate'].dt.week
df_italy.head()
#Visualizing Number of Confirmed cases , Deaths and Recovery in italy
fig1=px.line(df_italy,x='ObservationDate',y='Confirmed',title='Confirmed Cases in Italy')
fig2=px.line(df_italy,x='ObservationDate',y='Deaths',title='Deaths in Italy')
fig3=px.line(df_italy,x='ObservationDate',y='Recovered',title='Recovery in Italy')
fig4=px.line(df_italy,x='ObservationDate',y='ActiveCases',title='Active Cases in Italy')
fig1.show()
fig2.show()
fig3.show()
fig4.show()

In [ ]:
#Visualizing Cumaltive trends
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_italy['ObservationDate'], y=df_italy['Confirmed'],
                    mode='lines',name=' Confirmed Cases'))
fig.add_trace(go.Scatter(x=df_italy['ObservationDate'], y=df_italy['Deaths'], 
                mode='lines',name='Deaths'))
fig.add_trace(go.Scatter(x=df_italy['ObservationDate'], y=df_italy['Recovered'], 
                mode='lines',name='Recovered Cases'))
fig.add_trace(go.Scatter(x=df_italy['ObservationDate'], y=df_italy['ActiveCases'], 
                mode='lines',name='Active Cases'))

        
    
fig.update_layout(title_text='Trend in Italy',plot_bgcolor='rgb(250, 242, 242)')

fig.show()

In [ ]:
#Taking out record for latest records in seprate df
df_italy_lately=df_italy[df_italy['ObservationDate'] == latest_date]
#Creating temporary data frame for visualizing purpose
df_italy_status=pd.DataFrame(columns=['Numbers','Status'])
#Appending Records from latest data frame
df_italy_status=df_italy_status.append({'Numbers':df_italy_lately.iloc[0]['Confirmed'],'Status':'Confirmed'},ignore_index=True)
df_italy_status=df_italy_status.append({'Numbers':df_italy_lately.iloc[0]['ActiveCases'],'Status':'Active Cases'},ignore_index=True)
df_italy_status=df_italy_status.append({'Numbers':df_italy_lately.iloc[0]['Recovered'],'Status':'Recovered'},ignore_index=True)
df_italy_status=df_italy_status.append({'Numbers':df_italy_lately.iloc[0]['Deaths'],'Status':'Deaths'},ignore_index=True)
#Visualizing graphs
fig = px.bar(df_italy_status, y='Numbers', x='Status',title='Latest Status in Italy' , color='Numbers')
fig.show()

**📌Take away points**

* There is a sudden surge in confirmed cases in and after first week of March
* Rate by which confirmed cases are increasing is way higher than the rate of recovery and deaths , which is not in the case of China
* With in 10 days (between 12th-21st March) death tolls increased from **827** to **4825**

In [ ]:
#Calculating Death,Active Cases and Recovery percentage
death_percentage=round((df_italy_lately.iloc[0]['Deaths']/df_italy_lately.iloc[0]['Confirmed'])*100)
active_cases_percentage=round((df_italy_lately.iloc[0]['ActiveCases']/df_italy_lately.iloc[0]['Confirmed'])*100)
recovery_percentage=round((df_italy_lately.iloc[0]['Recovered']/df_italy_lately.iloc[0]['Confirmed'])*100)

print("In Italy Active Cases and Recovery percentage is {} ,{} respectively and Death rate is {} ".format(active_cases_percentage,recovery_percentage,death_percentage))

Situation in Italy is not at all good , there number of recovery is pretty bad and number of deaths keep on increasing

Let's see the number in South Korea

In [ ]:
#Taking out of data for SouthKorea region
df_SouthKorea=df[df['Country/Region'] == 'South Korea']
print("Number of Records for SouthKorea are {}".format(df_SouthKorea.shape))
df_SouthKorea.head()

Let's visualize individual trends in South Korea

In [ ]:
#Group by date
df_SouthKorea=df_SouthKorea.groupby(['ObservationDate']).sum().reset_index()
df_SouthKorea['Week'] = df_SouthKorea['ObservationDate'].dt.week

df_SouthKorea.head()
#Visualizing Number of Confirmed cases , Deaths and Recovery in SouthKorea
fig1=px.line(df_SouthKorea,x='ObservationDate',y='Confirmed',title='Confirmed Cases in South Korea')
fig2=px.line(df_SouthKorea,x='ObservationDate',y='Deaths',title='Deaths in South Korea')
fig3=px.line(df_SouthKorea,x='ObservationDate',y='Recovered',title='Recovery in South Korea')
fig4=px.line(df_SouthKorea,x='ObservationDate',y='ActiveCases',title='Active Cases in South Korea')

fig1.show()
fig2.show()
fig3.show()
fig4.show()

In [ ]:
#Visualizing Cumaltive trends
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_SouthKorea['ObservationDate'], y=df_SouthKorea['Confirmed'],
                    mode='lines',name=' Confirmed Cases'))
fig.add_trace(go.Scatter(x=df_SouthKorea['ObservationDate'], y=df_SouthKorea['Deaths'], 
                mode='lines',name='Deaths'))
fig.add_trace(go.Scatter(x=df_SouthKorea['ObservationDate'], y=df_SouthKorea['Recovered'], 
                mode='lines',name='Recovered Cases'))
fig.add_trace(go.Scatter(x=df_SouthKorea['ObservationDate'], y=df_SouthKorea['ActiveCases'], 
                mode='lines',name='Active Cases'))

        
    
fig.update_layout(title_text='Trend in South Korea',plot_bgcolor='rgb(250, 242, 242)')

fig.show()

In [ ]:
#Taking out latest records in seprate df
df_SouthKorea_lately=df_SouthKorea[df_SouthKorea['ObservationDate'] == latest_date]
#Creating temporary data frame for visualizing purpose
df_sk_status=pd.DataFrame(columns=['Numbers','Status'])
#Appending Records from latest data frame
df_sk_status=df_sk_status.append({'Numbers':df_SouthKorea_lately.iloc[0]['Confirmed'],'Status':'Confirmed'},ignore_index=True)
df_sk_status=df_sk_status.append({'Numbers':df_SouthKorea_lately.iloc[0]['ActiveCases'],'Status':'Active Cases'},ignore_index=True)
df_sk_status=df_sk_status.append({'Numbers':df_SouthKorea_lately.iloc[0]['Recovered'],'Status':'Recovered'},ignore_index=True)
df_sk_status=df_sk_status.append({'Numbers':df_SouthKorea_lately.iloc[0]['Deaths'],'Status':'Deaths'},ignore_index=True)
#Visualizing graphs
fig = px.bar(df_sk_status, y='Numbers', x='Status',title='Latest Status in SouthKorea' , color='Numbers')
fig.show()

**📌Take away points**

* Confirmed cases have increased from **31** to **602** with in 5 days i.e between Feburary 19 and Feburary 24 , this may be because of **Patient31** of South Korea (refer this [article](https://graphics.reuters.com/CHINA-HEALTH-SOUTHKOREA-CLUSTERS/0100B5G33SB/index.html) to read about Patient31)
* From 15th March rate of confirmation is kind of stable and not increased much as compare to pervious weeks 
* After 15th March rate of recovery has increased drastically and number of deaths is less as compared to confirmed cases

In [ ]:
#Calculating Death,Active Cases and Recovery percentage
death_percentage=round((df_SouthKorea_lately.iloc[0]['Deaths']/df_SouthKorea_lately.iloc[0]['Confirmed'])*100)
active_cases_percentage=round((df_SouthKorea_lately.iloc[0]['ActiveCases']/df_SouthKorea_lately.iloc[0]['Confirmed'])*100)
recovery_percentage=round((df_SouthKorea_lately.iloc[0]['Recovered']/df_SouthKorea_lately.iloc[0]['Confirmed'])*100)

print("In South Korea Active Cases and Recovery percentage is {} ,{} respectively and Death rate is {} ".format(active_cases_percentage,recovery_percentage,death_percentage))

Like China , South Korea also did a commendable job in controlling this pandemic which is evident with their low death rate but number of active cases are high which is close to **60%** . But unlike Italy , despite of sudden surge in cases they were able to handle them efficently.

> >Let's list down the points which would be ideal in current scenario
> * Rate of Confirmed Cases should not be increasing drastically
> * Rate of Death should be very low
> * Rate of Active cases should be gradually decreasing
> * Rate of Recovery should be gradually increasing
> 
> **It's evident from above graphs that China and South Korea are exhibiting all these points. Any country who are in early stages should exhibit these trends to fight against COVOID-19**

# What is the trend in USA and INDIA

Let's start with USA

In [ ]:
#Taking out of data for USA region
df_USA=df[df['Country/Region'] == 'US']
print("Number of Records for USA are {}".format(df_USA.shape))
df_USA.head()

Let's see the individual trend in USA

In [ ]:
#Group by date
df_USA=df_USA.groupby(['ObservationDate']).sum().reset_index()
df_USA['Week'] = df_USA['ObservationDate'].dt.week

df_USA.head()
#Visualizing Number of Confirmed cases , Deaths and Recovery in USA
fig1=px.line(df_USA,x='ObservationDate',y='Confirmed',title='Confirmed Cases in USA')
fig2=px.line(df_USA,x='ObservationDate',y='Deaths',title='Deaths in USA')
fig3=px.line(df_USA,x='ObservationDate',y='Recovered',title='Recovery in USA')
fig4=px.line(df_USA,x='ObservationDate',y='ActiveCases',title='Active Cases in USA')

fig1.show()
fig2.show()
fig3.show()
fig4.show()

> **Note**: US data for 'Recovered' is not correct for couple of days. I have posted this in discussion [thread](https://www.kaggle.com/sudalairajkumar/novel-corona-virus-2019-dataset/discussion/138705) ,but not recived any reposne

In [ ]:
#Visualizing Cumaltive trends
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_USA['ObservationDate'], y=df_USA['Confirmed'],
                    mode='lines',name=' Confirmed Cases'))
fig.add_trace(go.Scatter(x=df_USA['ObservationDate'], y=df_USA['Deaths'], 
                mode='lines',name='Deaths'))
fig.add_trace(go.Scatter(x=df_USA['ObservationDate'], y=df_USA['Recovered'], 
                mode='lines',name='Recovered Cases'))
fig.add_trace(go.Scatter(x=df_USA['ObservationDate'], y=df_USA['ActiveCases'], 
                mode='lines',name='Active Cases'))

        
    
fig.update_layout(title_text='Trend in USA',plot_bgcolor='rgb(250, 242, 242)')

fig.show()

In [ ]:
#Taking out latest records in seprate df
df_USA_lately=df_USA[df_USA['ObservationDate'] == latest_date]
#Creating temporary data frame for visualizing purpose
df_usa_status=pd.DataFrame(columns=['Numbers','Status'])
#Appending Records from latest data frame
df_usa_status=df_usa_status.append({'Numbers':df_USA_lately.iloc[0]['Confirmed'],'Status':'Confirmed'},ignore_index=True)
df_usa_status=df_usa_status.append({'Numbers':df_USA_lately.iloc[0]['ActiveCases'],'Status':'Active Cases'},ignore_index=True)
df_usa_status=df_usa_status.append({'Numbers':df_USA_lately.iloc[0]['Recovered'],'Status':'Recovered'},ignore_index=True)
df_usa_status=df_usa_status.append({'Numbers':df_USA_lately.iloc[0]['Deaths'],'Status':'Deaths'},ignore_index=True)
#Visualizing graphs
fig = px.bar(df_usa_status, y='Numbers', x='Status',title='Latest Status in USA' , color='Numbers')
fig.show()

**📌Take away points**

* There is huge surge in confirmed cases in US after March 10 , cases rises from **267** to **15,793** in 12 days(10 March-22 March)
* Death rates increased drastically from **1** to **117** with 22 days (1 March- 22 March) 
* Rate of recovery was kind of flat till 15 March after that it increased
* Majority of cases are still active 

In [ ]:
#Calculating Death,Active Cases and Recovery percentage
death_percentage=round((df_USA_lately.iloc[0]['Deaths']/df_USA_lately.iloc[0]['Confirmed'])*100)
active_cases_percentage=round((df_USA_lately.iloc[0]['ActiveCases']/df_USA_lately.iloc[0]['Confirmed'])*100)
recovery_percentage=round((df_USA_lately.iloc[0]['Recovered']/df_USA_lately.iloc[0]['Confirmed'])*100)

print("In US Active Cases and Recovery percentage is {} ,{} respectively and Death rate is {} ".format(active_cases_percentage,recovery_percentage,death_percentage))

>Rate of deaths looks very low but since active cases are huge death tolls could rise in USA

Let's check about India

In [ ]:
#Taking out of data for India region
df_India=df[df['Country/Region'] == 'India']
print("Number of Records for India are {}".format(df_India.shape))
df_India.head()

Let's visualize individual trend in India

In [ ]:
#Group by date
df_India=df_India.groupby(['ObservationDate']).sum().reset_index()
df_India['Week'] = df_India['ObservationDate'].dt.week

df_India.head()
#Visualizing Number of Confirmed cases , Deaths and Recovery in India
fig1=px.line(df_India,x='ObservationDate',y='Confirmed',title='Confirmed Cases in India')
fig2=px.line(df_India,x='ObservationDate',y='Deaths',title='Deaths in India')
fig3=px.line(df_India,x='ObservationDate',y='Recovered',title='Recovery in India')
fig4=px.line(df_India,x='ObservationDate',y='ActiveCases',title='Active Cases in India')

fig1.show()
fig2.show()
fig3.show()
fig4.show()

In [ ]:
#Visualizing Cumaltive trends
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_India['ObservationDate'], y=df_India['Confirmed'],
                    mode='lines',name=' Confirmed Cases'))
fig.add_trace(go.Scatter(x=df_India['ObservationDate'], y=df_India['Deaths'], 
                mode='lines',name='Deaths'))
fig.add_trace(go.Scatter(x=df_India['ObservationDate'], y=df_India['Recovered'], 
                mode='lines',name='Recovered Cases'))
fig.add_trace(go.Scatter(x=df_India['ObservationDate'], y=df_India['ActiveCases'], 
                mode='lines',name='Active Cases'))

fig.update_layout(title_text='Trend in India',plot_bgcolor='rgb(250, 242, 242)')

fig.show()

In [ ]:
#Taking out latest records in seprate df
df_India_lately=df_India[df_India['ObservationDate'] == latest_date]
#Creating temporary data frame for visualizing purpose
df_India_status=pd.DataFrame(columns=['Numbers','Status'])
#Appending Records from latest data frame
df_India_status=df_India_status.append({'Numbers':df_India_lately.iloc[0]['Confirmed'],'Status':'Confirmed'},ignore_index=True)
df_India_status=df_India_status.append({'Numbers':df_India_lately.iloc[0]['ActiveCases'],'Status':'Active Cases'},ignore_index=True)
df_India_status=df_India_status.append({'Numbers':df_India_lately.iloc[0]['Recovered'],'Status':'Recovered'},ignore_index=True)
df_India_status=df_India_status.append({'Numbers':df_India_lately.iloc[0]['Deaths'],'Status':'Deaths'},ignore_index=True)
#Visualizing graphs
fig = px.bar(df_India_status, y='Numbers', x='Status',title='Latest Status in India' , color='Numbers')
fig.show()

**📌Take away points**

* In India also number of confirmed cases increased drastically from first week of March , within 20 days it raised from **5** to **396** (2 March-22 March) but numbers are significantly low as compare to US
* Death and Recovery rate is relatively low 
* India also have huge number of Active cases


In [ ]:
#Calculating Death,Active Cases and Recovery percentage
death_percentage=round((df_India_lately.iloc[0]['Deaths']/df_India_lately.iloc[0]['Confirmed'])*100)
active_cases_percentage=round((df_India_lately.iloc[0]['ActiveCases']/df_India_lately.iloc[0]['Confirmed'])*100)
recovery_percentage=round((df_India_lately.iloc[0]['Recovered']/df_India_lately.iloc[0]['Confirmed'])*100)

print("In India Active Cases and Recovery percentage is {} ,{} respectively and Death rate is {} ".format(active_cases_percentage,recovery_percentage,death_percentage))

>Both USA and India is not doing in terms of cumulative numbers and graph , they have high number of Active cases and low number of recovery which is not at all good

# China/SouthKorea/Italy Vs USA/India

At last , I want to Compare confirmed cases and Active Cases in these countries across weeks to find out where US and India can land up in upcoming weeks

In [ ]:
#Visualizing trends across weeks
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_china['Week'], y=df_china['Confirmed'],
                    mode='lines+markers',name='China'))
fig.add_trace(go.Scatter(x=df_SouthKorea['Week'], y=df_SouthKorea['Confirmed'],
                    mode='lines+markers',name='South Korea'))
fig.add_trace(go.Scatter(x=df_italy['Week'], y=df_italy['Confirmed'],
                    mode='lines+markers',name='Italy'))
fig.add_trace(go.Scatter(x=df_USA['Week'], y=df_USA['Confirmed'],
                    mode='lines+markers',name='USA'))
fig.add_trace(go.Scatter(x=df_India['Week'], y=df_India['Confirmed'],
                    mode='lines+markers',name='India'))
fig.update_layout(title_text='Confirmed Cases Across Countries in Weeks',xaxis_title='Number of Weeks',yaxis_title='Number of Cases',plot_bgcolor='rgb(250, 242, 242)', font=dict(
        family="Courier New, monospace",
        size=18,
    ))

fig.show()

In [ ]:
#Visualizing trends across weeks
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_china['Week'], y=df_china['ActiveCases'],
                    mode='lines+markers',name='China'))
fig.add_trace(go.Scatter(x=df_SouthKorea['Week'], y=df_SouthKorea['ActiveCases'],
                    mode='lines+markers',name='South Korea'))
fig.add_trace(go.Scatter(x=df_italy['Week'], y=df_italy['ActiveCases'],
                    mode='lines+markers',name='Italy'))
fig.add_trace(go.Scatter(x=df_USA['Week'], y=df_USA['ActiveCases'],
                    mode='lines+markers',name='USA'))
fig.add_trace(go.Scatter(x=df_India['Week'], y=df_India['ActiveCases'],
                    mode='lines+markers',name='India'))
fig.update_layout(title_text='Active Cases Across Countries in Weeks',xaxis_title='Number of Weeks',yaxis_title='Number of Cases',plot_bgcolor='rgb(250, 242, 242)', font=dict(
        family="Courier New, monospace",
        size=18,
    ))

fig.show()

**📌Take away points**

* For China in intial weeks there is a surge in Confirmed Cases but after 4th week of reporting intial cases(#8 at x-axis in above graphs) active cases started reducing 
* Similarly , for South Korea after 4th week of reporting initial cases(#11 at x-axis in above graph) confirmed cases are not increasing much and active cases are declining slowly
* For US and Italy after 7th week of reporting initial cases(#11 at x-axis in above graph) there is sudden surge in confirmed cases and this is rapidly increasing week by week (We can see this by observing each **Week Steps Height** in above graphs ,which is quite high for US/Italy as compare to China/SouthKorea)
* As if now numbers are not high in India but here also there is a surge after after 7th week of reporting initial cases(#11 at x-axis in above graph) ,this is significantly very less as compare to Italy and US

# Last Word !!

Well, this is evident from the above analysis that China and South Korea can control and tackle COVID-19 efficiently as they have started all their measures early. Their active cases are coming down significantly after the 4th week of initially reported cases.

Whereas Italy and the US are following **Polynomial Regression** curve in Confirmed and Active cases which means in upcoming weeks it's going to increase for sure. There is very little chance of a decline in this curve, especially for the US because there is no complete country lockdown and the US government is only focusing on areas where the number of cases is more.

Also, there was a significant amount of time between initial cases and sudden surge (close to 7 weeks) but there was no proper measures and actions during this period and I strongly believe this may be the reason behind sudden surge because in absence of lockdown this time frame is sufficient enough to spread the virus.

As if now, fortunately, this is not happening for India as numbers are still less and if Indians follow 21 days complete lockdown (starting from 24 March till 14th April) rigorously then the situation could be better, hope for the best.

**Thoughts and Prayers with all who are fighting with COVID-19**